In [1]:
import tabulate

def print_data_table(dataset, exclude_columns=[]):
    dataset = [
        {key: value for key, value in entry.items() if key not in exclude_columns}
        for entry in dataset
    ]
    header = dataset[0].keys()
    rows = [x.values() for x in dataset]
    print(tabulate.tabulate(rows, header))

In [ ]:
from datetime import datetime

def get_avg_ttc_per_file(log_file_path):
    with open(log_file_path, 'r') as file:
        lines = file.readlines()

    valid_lines = []
    current_measurement = []
    
    # Process each line
    for line in lines:
        if '--- NEW MEASUREMENT ROUND ---' in line:
            for i in range(0, len(current_measurement), 2):  # Step by 2 to ensure pairs
                if i+1 < len(current_measurement):
                    valid_lines.append(current_measurement[i])
                    valid_lines.append(current_measurement[i+1])
            current_measurement = []  # Reset for the next round
            continue
        
        # Extract value from the line
        try:
            value = line.split(' ')
            value = int(value.strip())
        except ValueError:
            continue
        
        current_measurement.append(line)
    
    # Handle last round of measurements
    for i in range(0, len(current_measurement), 2):
        if i+1 < len(current_measurement):
            valid_lines.append(current_measurement[i])
            valid_lines.append(current_measurement[i+1])

    time_diff_sum = 0
    counter = 0
    for i in range(0, len(valid_lines)):
        parts = valid_lines[i].split(' ')
        timestamp_str_1, data_value_str_1 = parts[0], parts[1]

        if i + 1 < len(valid_lines):
            parts = valid_lines[i + 1].split(' ')
            timestamp_str_2, data_value_str_2 = parts[0], parts[1]
        else:
            data_value_str_2 = None

        if data_value_str_1 == data_value_str_2:
            try:
                timestamp_1 = datetime.fromisoformat(timestamp_str_1.strip())
                timestamp_2 = datetime.fromisoformat(timestamp_str_2.strip())

                ttc = (timestamp_2 - timestamp_1).total_seconds()

                time_diff_sum += ttc
                counter += 1
            except ValueError:
                continue  # Skip malformed lines, if any
        else:
            continue

    return time_diff_sum / counter


In [ ]:
import os
import statistics

REPETITIONS = 30

ttc_data = []

exp_names = [
    'both_funcs_on_rpi',
    'both_funcs_on_vm',
    'first_func_on_rpi_second_func_on_vm',
    'first_func_on_vm_second_func_on_rpi'
]

for exp in exp_names:
    exp_log_dir = exp + '/ttc/'
    exp_ttc_data = {
        "exp_name": exp,
        "ttc_log_dir": exp_log_dir,
        "log_files": [os.path.join(exp_log_dir, f) for f in os.listdir(exp_log_dir) if f.endswith(".log")],
        "avg_ttc": None,
        "avg_ttc_measurements_per_file": []             # For the box plot
    }
    ttc_data.append(exp_ttc_data)


for exp in ttc_data:
    avg_ttc_per_file_list = []
    for log_file in exp['log_files']:
        avg_ttc_per_file_list.append(get_avg_ttc_per_file(log_file))
    exp['avg_ttc'] = statistics.mean(avg_ttc_per_file_list)
    exp['avg_ttc_measurements_per_file'] = avg_ttc_per_file_list


print_data_table(ttc_data, exclude_columns=['ttc_log_dir', 'log_files'])


all_data_lists = []
for exp in ttc_data:
    all_data_lists.append(exp['avg_ttc_measurements_per_file'])

In [ ]:
import json

# Build a minimal JSON-serializable list
json_export_data = [
    {
        "exp_name": exp["exp_name"],
        "avg_ttc": exp["avg_ttc"]
    }
    for exp in ttc_data
]

# Write to file
with open("summary_avg_ttc.json", "w") as f:
    json.dump(json_export_data, f, indent=4)

print("\nJSON summary written to summary_avg_ttc.json")

In [ ]:
import json

data = {
    '3_cores': [
        {
            'exp_name': ttc_data[0].get('exp_name'),
            'avg_ttc': ttc_data[0].get('avg_ttc')
        },
        {
            'exp_name': ttc_data[1].get('exp_name'),
            'avg_ttc': ttc_data[1].get('avg_ttc')
        },
        {
            'exp_name': ttc_data[2].get('exp_name'),
            'avg_ttc': ttc_data[2].get('avg_ttc')
        },
        {
            'exp_name': ttc_data[3].get('exp_name'),
            'avg_ttc': ttc_data[3].get('avg_ttc')
        },
    ]
}

json_str = json.dumps(data)
json_str

In [ ]:
import matplotlib.pyplot as plt

def show_combined_box_plot(x_labels, y_label, title, data):
    plt.boxplot(data, patch_artist=True, labels=x_labels)
    plt.yscale("log")
    plt.title(title)
    plt.ylabel(y_label)
    plt.show()

show_combined_box_plot(
    [
        'RPI\nonly', 
        'Two functions\non VMpub',
        'Feature\nextraction\non RPI,\nclassification\non VMpub',
        'Feature\nextraction\non VMpub,\nclassification\non RPI'
    ], 
    'TtC (s)',
    'Time to Complete (TtC) data (3 active cores)', 
    all_data_lists
)